In [1]:
# importing libaries

import pandas as pd
import numpy as np
import re

In [2]:
# importing data

df = pd.read_excel(r'C:\Users\CharlesYi\Jupyter Notebook\Alteryx Challenges\Challenge 386_Shed Light on a Banks Risky Business\Bank_Data.xlsx')

In [3]:
df.head()

,Reporting Month,KRI,Description,Location,Value,Threshold Green,Threshold Amber,Threshold Red,Green Low,Green High,Green Sign,Amber_low_sign,Amber Low,Amber_high_sign,Amber High,Red_sign,Red
0,2023-01-01,KRI1234,Number of currently open card transaction disp...,UK,298.0,0-300,>300 and <=800,>800,0,300.0,NaN,>,300.0,<=,800.0,>,800.0
1,2023-02-01,KRI1234,Number of currently open card transaction disp...,UK,308.0,0-300,>300 and <=1000,>1000,0,300.0,NaN,>,300.0,<=,1000.0,>,1000.0
2,2023-03-01,KRI1234,Number of currently open card transaction disp...,UK,340.0,0-350,>300 and <=1000,>1000,0,350.0,NaN,>,350.0,<=,1000.0,>,1000.0
3,2023-01-01,KRI5678,Number of stolen cards reported.,UK,220.0,0-100,>100 and <=200,>200,0,100.0,NaN,>,100.0,<=,200.0,>,200.0
4,2023-02-01,KRI5678,Number of stolen cards reported.,UK,180.0,0-100,>100 and <=200,>200,0,100.0,NaN,>,100.0,<=,200.0,>,200.0


In [16]:
# creating function to determine RAG

def RAG_func (df):
    
    # setting variables
    value = df['Value']
    green_low = df['Green Low']
    green_high = df['Green High'].fillna(999999)
    amber_low = df['Amber Low']
    amber_high = df['Amber High']
    red_threshold = df['Red']
    red_sign = df['Red_sign']
    
    conditions = [(value > green_low) & (value <= green_high),
                  (value > amber_low) & (value <= amber_high),
                  (red_sign == '>') & (value > amber_high) & (value < amber_high * 10000),
                  (red_sign == '<') & (value < red_threshold)]
    
    choices = ['Green', 'Amber', 'Red', 'Red']
    
    df['RAG'] = np.select(conditions, choices, default = 'unknown')
    
    return df

In [18]:
# applying function

df_rag = RAG_func(df)

df_rag.head()

,Reporting Month,KRI,Description,Location,Value,Threshold Green,Threshold Amber,Threshold Red,Green Low,Green High,Green Sign,Amber_low_sign,Amber Low,Amber_high_sign,Amber High,Red_sign,Red,RAG
0,2023-01-01,KRI1234,Number of currently open card transaction disp...,UK,298.0,0-300,>300 and <=800,>800,0,300.0,NaN,>,300.0,<=,800.0,>,800.0,Green
1,2023-02-01,KRI1234,Number of currently open card transaction disp...,UK,308.0,0-300,>300 and <=1000,>1000,0,300.0,NaN,>,300.0,<=,1000.0,>,1000.0,Amber
2,2023-03-01,KRI1234,Number of currently open card transaction disp...,UK,340.0,0-350,>300 and <=1000,>1000,0,350.0,NaN,>,350.0,<=,1000.0,>,1000.0,Green
3,2023-01-01,KRI5678,Number of stolen cards reported.,UK,220.0,0-100,>100 and <=200,>200,0,100.0,NaN,>,100.0,<=,200.0,>,200.0,Red
4,2023-02-01,KRI5678,Number of stolen cards reported.,UK,180.0,0-100,>100 and <=200,>200,0,100.0,NaN,>,100.0,<=,200.0,>,200.0,Amber


In [25]:
# selecting relevant columns

df_rag[['Reporting Month', 'KRI', 'Description', 'RAG']].sort_values(by =['KRI', 'Reporting Month'])

,Reporting Month,KRI,Description,RAG
0,2023-01-01,KRI1234,Number of currently open card transaction disp...,Green
1,2023-02-01,KRI1234,Number of currently open card transaction disp...,Amber
2,2023-03-01,KRI1234,Number of currently open card transaction disp...,Green
9,2023-01-01,KRI1657,Number of forged checks and drafts presented.,Amber
10,2023-02-01,KRI1657,Number of forged checks and drafts presented.,Amber
11,2023-03-01,KRI1657,Number of forged checks and drafts presented.,Amber
15,2023-01-01,KRI4634,Percentage of staff who haven’t completed prim...,Green
16,2023-02-01,KRI4634,Percentage of staff who haven’t completed prim...,Amber
17,2023-03-01,KRI4634,Percentage of staff who haven’t completed prim...,Green
3,2023-01-01,KRI5678,Number of stolen cards reported.,Red
